In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as io

import os

In [2]:
KIN_FACE_WI = '.\\data\\KinFaceW\\KinFaceW-I'
KIN_FACE_WII = '.\\data\\KinFaceW\\KinFaceW-II'

In [3]:
def get_paths(base_path):
    return {
        'fd': base_path + '\\meta_data\\fd_pairs.mat',
        'fs': base_path + '\\meta_data\\fs_pairs.mat',
        'md': base_path + '\\meta_data\\md_pairs.mat',
        'ms': base_path + '\\meta_data\\ms_pairs.mat'
    }

In [4]:
def load_mat_file(paths):
    fd, fs, md, ms = paths['fd'], paths['fs'], paths['md'], paths['ms']
    
    return {
        'fd': io.loadmat(fd),
        'fs': io.loadmat(fs),
        'md': io.loadmat(md),
        'ms': io.loadmat(ms)
    }

In [5]:
def build_df(mats):
    folds = []
    labels = []
    firsts = []
    seconds = []
    rels = []
    
    fd_mat, fs_mat, md_mat, ms_mat = mats['fd'], mats['fs'], mats['md'], mats['ms']
    rel_list = ['father_daughter', 'father_son', 'mother_daughter', 'mother_son']
    mats = [fd_mat, fs_mat, md_mat, ms_mat]

    for i in range(4):
        for pair in mats[i]['pairs']:
            fold, label, first, second = pair

            folds.append(fold[0][0])
            labels.append(label[0][0])
            firsts.append(first[0])
            seconds.append(second[0])
            rels.append(rel_list[i])
    
    df = pd.DataFrame.from_dict({
    'first': firsts,
    'second': seconds,
    'label': labels,
    'fold': folds,
    'rel': rels
    
    })
    
    return df

In [9]:
def split_folds(df):
    images = {}
    for i in range(1, 5):
        fold_data = df.loc[df['fold'] == i]
        first = set(fold_data['first'])
        second = set(fold_data['second'])
        
        total = first.union(second)
        images[i] = total
        
    return images

In [7]:
def build_file(root, folds):
    with open(root, 'w') as file:
        for i in range(1, 5):
            for item in folds[i]:
                file.write(f'{item} ')
            file.write('\n')

In [10]:
df = build_df(load_mat_file(get_paths(KIN_FACE_WI)))
images = split_folds(df)
build_file('KinFaceWIFolds.txt', images)

df = build_df(load_mat_file(get_paths(KIN_FACE_WII)))
images = split_folds(df)
build_file('KinFaceWIIFolds.txt', images)